In [1]:
import pandas as pd
import numpy as np
import re
import string
import emoji
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
import pickle

#for pickle a countvectorizer with lambda function
import dill

In [3]:
import warnings
warnings.filterwarnings('ignore')

## **Load Training Data**

Wisesight Dataset = 'https://docs.google.com/spreadsheets/d/1F_qT33T2iy0tKbflnVC8Ma-EoWEHimV3NmNRgLjN00o/edit#gid=1302375309'

In [4]:
df = pd.read_csv('WiseSight Tokenisation Annotation - WiseSight (250 samples_class).csv', sep=',')
df

,label,raw
0,q,ขอนุญาตินอกเรื่องกูข้องใจนานละ เมื่อไหร่sextoy...
1,q,เพอริเพอร่าไปยังค่ะ
2,q,ผมซื้อวีออส 2006 มือสองมา ต้องดูแลอะไรเป็นพิเศ...
3,q,เอานิสสันมาคร์เทรินได้ไหมเหลืออีกแสนนิดๆ
4,q,หุ้มพวงมาลัย มาสด้า 3 เท่าไรครับ
...,...,...
995,neg,ถ้าฉันเป็นแม่คุณแล้วรู้ว่าประหยัดผ้าอนามัยเก้า...
996,neg,แสงโสมแบนเดียวทำอะไรกูไม่ได้
997,neg,เหนื่อยใจแทน กะหน่วยงานในประเทศนี้...ตอแหล บิด...
998,neg,ไม่กินเผ็ดอะ


# **Data Cleaning**

In [5]:
should_removed = ~df.label.apply(lambda x: len(x.split("-")) > 1)
data_for_training = df[should_removed]
print("we have %d after samples" % len(data_for_training))

we have 993 after samples


In [6]:
#thai stopwords
thai_stopwords = list(thai_stopwords())

In [7]:
def clean_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

#ลบคำ เช่น ไวววววววววว ให้เป็น ไว
#(.) means แทนตัวไหนก็ได้
#\1 means first group
#{3,} means ซ้ำมากกว่า 3 ตัว
def clean_rep(text):
    return re.compile(r'(.)\1{3,}', re.IGNORECASE).sub(r'\1', text)

def remove_hashtag(text):
    return re.sub("#(\w+)", '', text)
    
#ลบคำ เช่น ไวไวไวไวไว ให้เป็น ไว
def remove_duplicate(text):
    return re.sub(r'(.+?)\1+', r'\1', text)

def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

def clean_text(text):
    text = text.lower().strip()
    text = clean_url(text)
    text = remove_hashtag(text)
    text = remove_punctuation(text)
    text = clean_rep(text)
    text = remove_duplicate(text)
    text = remove_emoji(text)
    
    #tokenize
    text = word_tokenize(text)
    text = " ".join(word for word in text)
    text = " ".join(word for word in text.split() if word.lower not in thai_stopwords)
    
    return text

In [8]:
data_for_training['text_tokens'] = data_for_training['raw'].apply(clean_text)
data_for_training

,label,raw,text_tokens
0,q,ขอนุญาตินอกเรื่องกูข้องใจนานละ เมื่อไหร่sextoy...,ขอ นุ ญาติ นอกเรื่อง กู ข้องใจ นาน ละ เมื่อไหร...
1,q,เพอริเพอร่าไปยังค่ะ,เพ อริ เพ อ ร่า ไป ยัง ค่ะ
2,q,ผมซื้อวีออส 2006 มือสองมา ต้องดูแลอะไรเป็นพิเศ...,ผม ซื้อ วี อส 206 มือสอง มา ต้อง ดูแล อะไร เป็...
3,q,เอานิสสันมาคร์เทรินได้ไหมเหลืออีกแสนนิดๆ,เอา นิ สัน มา คร์ เท ริน ได้ ไหม เหลือ ีก แส นิดๆ
4,q,หุ้มพวงมาลัย มาสด้า 3 เท่าไรครับ,หุ้ม พวงมาลัย มาสด้า 3 เท่าไร ครับ
...,...,...,...
995,neg,ถ้าฉันเป็นแม่คุณแล้วรู้ว่าประหยัดผ้าอนามัยเก้า...,ถ้า ฉัน เป็น แม่คุณ แล้ว รู้ ว่า ประหยัด ผ้าอน...
996,neg,แสงโสมแบนเดียวทำอะไรกูไม่ได้,แสงโสม แบน เดียว ทำ อะไร กู ไม่ ได้
997,neg,เหนื่อยใจแทน กะหน่วยงานในประเทศนี้...ตอแหล บิด...,เหนื่อยใจ แทน กะ หน่วยงาน ในประเทศ นี้ ตอแหล บ...
998,neg,ไม่กินเผ็ดอะ,ไม่ กิน เผ็ด อะ


# **Training Count Vectorizor & Sentiment Analyzer**

In [9]:
#Train, test, split the data

X = data_for_training[['text_tokens']]
y = data_for_training['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [10]:
#Training CountVetorizer to use for converting a collection of text documents to a matrix of token counts

cvec = CountVectorizer(analyzer=lambda x:x.split(' '))
cvec.fit_transform(X_train['text_tokens'])
cvec.vocabulary_

{'มันดี': 1756,
 'ปะ': 1534,
 'อ่ะ': 2480,
 'คือ': 797,
 'เห็น': 2953,
 'ตลับ': 1100,
 'มัน': 1755,
 'อิน': 2445,
 'สปาย': 2152,
 'จาก': 878,
 'tarte': 434,
 'เลย': 2875,
 'มะ': 1747,
 'ค่อ': 814,
 'ยาก': 1846,
 'ซื้อ': 1003,
 '5': 80,
 'เคย': 2593,
 'ใช้': 3237,
 'รุ่น': 1961,
 'แมท': 3080,
 'สี': 2239,
 'ดี': 1055,
 'อยู่': 2396,
 'แต่': 3026,
 'ก็': 608,
 'เป็น': 2772,
 'ก้อน': 621,
 'รุ่': 1960,
 'นี้': 1384,
 '้': 3344,
 'รีวิว': 1952,
 'ห้าม': 2379,
 'ขายของ': 660,
 'โว้ย': 3218,
 'เค': 2592,
 'ร': 1875,
 'ขอบ': 643,
 '14': 19,
 'ชุบ': 963,
 'กว้าง': 535,
 '65': 91,
 'ชุ': 961,
 'ป': 1482,
 'ใบ': 3249,
 'เท่าไร': 2726,
 'ครับ': 718,
 'ล้อรถ': 2052,
 'ย': 1830,
 'นิ': 1370,
 'สัน': 2203,
 'ผใ': 1608,
 'ตอ': 1102,
 'ปิด': 1551,
 'บริการ': 1427,
 'น้า': 1411,
 'กล่อง': 528,
 'คสอ': 766,
 'nyx': 339,
 'ยังมี': 1842,
 'ของ': 635,
 'มั้ย': 1763,
 'คะ': 779,
 'งาน': 834,
 'เข้า': 2585,
 'มาสด้า': 1783,
 'ฟ้อง': 1718,
 'ลูกค้า': 2037,
 'เรียกค่าเสียหาย': 2859,
 '95': 106,
 'ล้าน': 2057,


In [11]:
train_bow = cvec.transform(X_train['text_tokens'])
pd.DataFrame(train_bow.toarray(), columns=cvec.get_feature_names(), index=X_train['text_tokens'])

,,01,02,04,06,07,08,1,10,101,...,’,“,“ชัชา,”,…,─,╱,╱♡̷,ㅠ,️พอ
text_tokens,,,,,,,,,,,,,,,,,,,,,
มันดี ปะ อ่ะ คือ เห็น ตลับ มัน อิน สปาย จาก tarte เลย มะ ค่อ ยาก ซื้อ 5 เคย ใช้ รุ่น แมท สี ดี อยู่ แต่ ก็ เป็น ก้อน อ่ะ รุ่ นี้ มันดี มะ ้ รีวิว ห้าม ขายของ โว้ย,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
เค ร 5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ขอบ 14 ชุบ กว้าง 65 ชุ ป ใบ เท่าไร ครับ ล้อรถ ย นิ สัน ครับ ผใ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ตอ นี้ ปิด บริการ น้า,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
กล่อง คสอ nyx ยังมี ของ มั้ย คะ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ยอม ใจ พี่ นิ สัน รถ ราคา 5 แสน แต่ ราคา ทะเบียน ซื้อ รถ ข้างๆ ได้ 3 คัน เลย ทีเดียว,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ยัง กล้า กิน อยู่ รึ ป่าว,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Using the above BOW to train a LogisticRegression for a Sentiment Analyzer

lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(train_bow, y_train)

LogisticRegression(max_iter=1000)

In [13]:
#Test Score

test_bow = cvec.transform(X_test['text_tokens'])
test_predictions = lr.predict(test_bow)
print(classification_report(test_predictions, y_test))

              precision    recall  f1-score   support

         neg       0.55      0.68      0.61        72
         neu       0.39      0.43      0.41        60
         pos       0.65      0.49      0.56        82
           q       0.80      0.76      0.78        84

    accuracy                           0.60       298
   macro avg       0.60      0.59      0.59       298
weighted avg       0.61      0.60      0.60       298



In [14]:
with open('sentiment_model_pkl', 'wb') as pickle_file:
    pickle.dump(lr, pickle_file)

In [15]:
with open('countvectorizer_dill', 'wb') as dill_file:
    dill.dump(cvec, dill_file)

# **Testing CountVectorizor & Sentiment Analyzer**

In [16]:
with open('sentiment_model_pkl', 'rb') as s:
    model = pickle.load(s)

In [17]:
with open('countvectorizer_dill', 'rb') as c:
    loaded_cvec = dill.load(c)

In [18]:
def sentiment_prediction(text):
    cleaned_text = clean_text(text)
    bow = loaded_cvec.transform(pd.Series([cleaned_text]))
    prediction = model.predict(bow)[0]
    return prediction

In [19]:
sentiment_prediction('ไม่เป็นไร เธอทำดีแล้ว')

'pos'